<div style="max-width:1400px;margin-center: auto">
<img src="images\constructor standing.png" width="600"/>
</div>

In [1]:
import findspark
findspark.init()
from pyspark import SparkContext, SparkConf
from pyspark.sql import SparkSession
conf = SparkConf().setAppName('Constructor Standings').setMaster('local')
sc= SparkContext(conf=conf)
spark = SparkSession(sc)

In [4]:
result_df = spark.read.parquet(r'E:\unused\Udemy\Spark_practice\raw\presentation\race_result')
result_df.show(2)

+---------+-----------------+-------------------+----------------+--------------+-------------+------------------+-------+----+-----------+---------+------+--------+
|race_year|        race_name|          race_date|circuit_location|   driver_name|driver_number|driver_nationality|   team|grid|fastest_lap|race_time|points|position|
+---------+-----------------+-------------------+----------------+--------------+-------------+------------------+-------+----+-----------+---------+------+--------+
|     1974|German Grand Prix|1974-08-04 00:00:00|         Nürburg|Clay Regazzoni|         null|             Swiss|Ferrari|   2|       null|1:41:35.0|   9.0|       1|
|     1974|German Grand Prix|1974-08-04 00:00:00|         Nürburg|Jody Scheckter|         null|     South African|Tyrrell|   4|       null|    +50.7|   6.0|       2|
+---------+-----------------+-------------------+----------------+--------------+-------------+------------------+-------+----+-----------+---------+------+--------+
only

In [10]:
from pyspark.sql.functions import rank,desc, sum, count, col, when
from pyspark.sql.window import Window

In [14]:
constructor_standing = result_df.\
            groupBy('team','race_year').\
            agg(sum('points').alias('Total Points'),
               count(when(col("position") == 1, True)).alias('wins'))
constructor_standing.filter('race_year = 2020').show(2)

+--------+---------+------------+----+
|    team|race_year|Total Points|wins|
+--------+---------+------------+----+
| McLaren|     2020|       202.0|   0|
|Williams|     2020|         0.0|   0|
+--------+---------+------------+----+
only showing top 2 rows



In [20]:
constructorWindowSpec = Window.partitionBy('race_year').orderBy(desc('Total Points'), desc('wins'))
constructor_standings = constructor_standing.withColumn('rank', rank().over(constructorWindowSpec))
constructor_standings.filter('race_year = 2020').show(11)

+------------+---------+------------+----+----+
|        team|race_year|Total Points|wins|rank|
+------------+---------+------------+----+----+
|    Mercedes|     2020|       573.0|  13|   1|
|    Red Bull|     2020|       319.0|   2|   2|
|Racing Point|     2020|       210.0|   1|   3|
|     McLaren|     2020|       202.0|   0|   4|
|     Renault|     2020|       181.0|   0|   5|
|     Ferrari|     2020|       131.0|   0|   6|
|  AlphaTauri|     2020|       107.0|   1|   7|
|  Alfa Romeo|     2020|         8.0|   0|   8|
|Haas F1 Team|     2020|         3.0|   0|   9|
|    Williams|     2020|         0.0|   0|  10|
+------------+---------+------------+----+----+



In [22]:
constructor_standings.write.parquet('presentation/constructor standing')